In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pandas_datareader as pdr
from scipy.optimize import minimize
import json

argv = ["function_name", "MSFT AAPL GOOG AMZN XOM HSBC BRK-B JPM BAC WFC ", "200", "100000"]

In [3]:
%%time
'''
Based on : https://stackoverflow.com/questions/23450534/how-to-call-a-python-function-from-node-js
argv[1] : String of valid stock tickers in the portfolio
argv[2] : Duration window (in days) for stock variance calculations
argv[3] : Portfolio Capital (in cents)
'''
stocks = argv[1].split()
window = int(argv[2])
initial_capital = int(argv[3])

# Get data
end_date = datetime.today()
start_date = end_date - timedelta(window+2)
# stocks = ['MSFT','AAPL','GOOG','AMZN','XOM','HSBC','BRK-B','JPM','BAC','WFC']
stock_data = pdr.get_data_yahoo(symbols=stocks, start=start_date, end=end_date)
stock_data = stock_data.dropna()

# Calculate returns
for ticker in stock_data.columns.levels[1]:
    stock_data['Return', ticker] = stock_data['Adj Close', ticker].pct_change(1)
    
stock_data = stock_data.dropna()

tickers = []
invVar = []

# Calculate Inverse Variance
for ticker in stock_data.columns.levels[1]:
    tickers.append(ticker)
    invVar.append(stock_data['Return', ticker].var())

weights = invVar/sum(invVar)

portfolio_weights = dict()

for index, ticker in enumerate(tickers):
    portfolio_weights[ticker] = weights[index]
    
print(json.dumps(portfolio_weights))

{"AAPL": 0.14555224008282974, "AMZN": 0.09771034995307251, "BAC": 0.12830530758767827, "BRK-B": 0.04538362656218139, "GOOG": 0.16229699716716003, "HSBC": 0.05586733694802122, "JPM": 0.09241090109745582, "MSFT": 0.0967305951623466, "WFC": 0.10257457541906366, "XOM": 0.07316807002019061}
Wall time: 10.1 s


In [4]:
%%time
'''
Based on : https://stackoverflow.com/questions/23450534/how-to-call-a-python-function-from-node-js
argv[1] : String of valid stock tickers in the portfolio
argv[2] : Duration window (in days) for stock variance calculations
argv[3] : Portfolio Capital (in cents)
'''
stocks = argv[1].split()
window = int(argv[2])
initial_capital = int(argv[3])

# Get data
end_date = datetime.today()
start_date = end_date - timedelta(window+2)
# stocks = ['MSFT','AAPL','GOOG','AMZN','XOM','HSBC','BRK-B','JPM','BAC','WFC']
stock_data = pdr.get_data_yahoo(symbols=stocks, start=start_date, end=end_date)
stock_data = stock_data.dropna()

# Calculate returns
for ticker in stock_data.columns.levels[1]:
    stock_data['Return', ticker] = stock_data['Adj Close', ticker].pct_change(1)
    
stock_data = stock_data.dropna()
# print(stock_data)
cov = stock_data.Return.cov()
# print(covariance)
tickers = list(cov.columns.values)


# Optimization
def cost(w):
    global cov
    portfolio_variance = np.dot(np.dot(w.T, cov), w)
    return portfolio_variance
 
bnds = [(0.0001, 0.9999)]*len(cov)
cons = ({'type': 'eq', 'fun': lambda w: 1.0 - sum(w)})
w0 = np.random.randn(len(cov))/len(cov)

result = minimize(cost, w0, method='SLSQP', bounds=bnds, constraints=cons, tol=1e-6)
# print(result.message)

portfolio_weights = dict()

if result.success:
    weights = result.x
    for index, ticker in enumerate(tickers):
        portfolio_weights[ticker] = weights[index]

    
print(json.dumps(portfolio_weights))

{"AAPL": 0.09608119518937858, "AMZN": 0.12024337873278691, "BAC": 0.08093521958372155, "BRK-B": 0.08652442358593625, "GOOG": 0.10521060368077756, "HSBC": 0.12427681352438555, "JPM": 0.08094426398108054, "MSFT": 0.08094419276008524, "WFC": 0.1354697376646824, "XOM": 0.08937017129716596}
Wall time: 9.65 s


In [31]:
%%time
'''
Based on : https://stackoverflow.com/questions/23450534/how-to-call-a-python-function-from-node-js
argv[1] : String of valid stock tickers in the portfolio
argv[2] : Duration window (in days) for stock variance calculations
argv[3] : Portfolio Capital (in cents)
'''
stocks = argv[1].split()
window = int(argv[2])
initial_capital = int(argv[3])

# Get data
end_date = datetime.today()
start_date = end_date - timedelta(window+2)
# stocks = ['MSFT','AAPL','GOOG','AMZN','XOM','HSBC','BRK-B','JPM','BAC','WFC']
stock_data = pdr.get_data_yahoo(symbols=stocks, start=start_date, end=end_date)
stock_data = stock_data.dropna()

# Calculate returns
for ticker in stock_data.columns.levels[1]:
    stock_data['Return', ticker] = stock_data['Adj Close', ticker].pct_change(1)
    
stock_data = stock_data.dropna()
tickers = list(stock_data.columns.levels[1])

cov = np.array(stock_data.Return.cov())
exp_ret = 1.0 / np.array(stock_data.Return.mean())


max_exp_ret = np.max(np.abs(exp_ret))
max_var = np.max(np.abs(cov))

print(tickers, exp_ret)

# Optimization
def cost(w):
    global cov, exp_ret, max_exp_ret, max_var
    portfolio_variance = np.dot(np.dot(w.T, cov), w)
    portfolio_expt_ret = np.dot(w.T, exp_ret)
#     print(portfolio_variance / max_var + portfolio_expt_ret / max_exp_ret)
    return portfolio_variance / max_var + portfolio_expt_ret / max_exp_ret
 
bnds = [(0.0001, 0.9999)]*len(cov)
cons = ({'type': 'eq', 'fun': lambda w: 1.0 - sum(w)})
w0 = np.random.randn(len(cov))/len(cov)

result = minimize(cost, w0, method='SLSQP', bounds=bnds, constraints=cons, tol=1e-6)
# print(result.message)

portfolio_weights = dict()

if result.success:
    weights = result.x
    for index, ticker in enumerate(tickers):
        portfolio_weights[ticker] = weights[index]

    
print(json.dumps(portfolio_weights))

{"AAPL": 0.00010000000000009532, "AMZN": 0.17803120087222238, "BAC": 0.00010000000000012955, "BRK-B": 0.0001, "GOOG": 0.00010000000000056258, "HSBC": 0.6062068630653246, "JPM": 0.0001, "MSFT": 0.0001, "WFC": 0.00010000000000000045, "XOM": 0.2150619360624525}
